In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import math
import copy


from datasets import load_dataset

from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, BertTokenizer

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
class LayerNormalization(nn.Module):

    def __init__(self, features: int, eps:float=10**-6) -> None:
        super().__init__()
        self.eps = eps
        self.alpha = nn.Parameter(torch.ones(features)) # alpha is a learnable parameter
        self.bias = nn.Parameter(torch.zeros(features)) # bias is a learnable parameter

    def forward(self, x):
        # x: (batch, seq_len, hidden_size)
         # Keep the dimension for broadcasting
        x =  x.float()
        mean = x.mean(dim = -1, keepdim = True) # (batch, seq_len, 1)
        # Keep the dimension for broadcasting
        std = x.std(dim = -1, keepdim = True) # (batch, seq_len, 1)
        # eps is to prevent dividing by zero or when std is very small

        return self.alpha * (x - mean) / (std + self.eps) + self.bias

class FeedForwardBlock(nn.Module):

    def __init__(self, d_model: int, d_ff: int, dropout: float) -> None:
        super().__init__()
        self.linear_1 = nn.Linear(d_model, d_ff) # w1 and b1
        self.dropout = nn.Dropout(dropout)
        self.linear_2 = nn.Linear(d_ff, d_model) # w2 and b2

    def forward(self, x):
        # (batch, seq_len, d_model) --> (batch, seq_len, d_ff) --> (batch, seq_len, d_model)
        return self.linear_2(self.dropout(torch.relu(self.linear_1(x))))

class InputEmbeddings(nn.Module):

    def __init__(self, d_model: int, vocab_size: int) -> None:
        super().__init__()
        self.d_model = d_model
        self.vocab_size = vocab_size
        self.embedding = nn.Embedding(vocab_size, d_model)

    def forward(self, x):
        # (batch, seq_len) --> (batch, seq_len, d_model)
        # Multiply by sqrt(d_model) to scale the embeddings according to the paper
        return self.embedding(x) * math.sqrt(self.d_model)

class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, seq_len: int, dropout: float) -> None:
        super().__init__()
        self.d_model = d_model
        self.seq_len = seq_len
        self.dropout = nn.Dropout(dropout)
        # Create a matrix of shape (seq_len, d_model)
        pe = torch.zeros(seq_len, d_model)
        # Create a vector of shape (seq_len)
        position = torch.arange(0, seq_len, dtype=torch.float).unsqueeze(1) # (seq_len, 1)
        # Create a vector of shape (d_model)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model)) # (d_model / 2)
        # Apply sine to even indices
        pe[:, 0::2] = torch.sin(position * div_term) # sin(position * (10000 ** (2i / d_model))
        # Apply cosine to odd indices
        pe[:, 1::2] = torch.cos(position * div_term) # cos(position * (10000 ** (2i / d_model))
        # Add a batch dimension to the positional encoding
        pe = pe.unsqueeze(0) # (1, seq_len, d_model)
        # Register the positional encoding as a buffer
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + (self.pe[:, :x.shape[1], :]).requires_grad_(False) # (batch, seq_len, d_model)
        return self.dropout(x)

class ResidualConnection(nn.Module):

        def __init__(self, features: int, dropout: float) -> None:
            super().__init__()
            self.dropout = nn.Dropout(dropout)
            self.norm = LayerNormalization(features)

        def forward(self, x, sublayer):
            return x + self.dropout(sublayer(self.norm(x)))

class MultiHeadAttentionBlock(nn.Module):

    def __init__(self, d_model: int, h: int, dropout: float) -> None:
        super().__init__()
        self.d_model = d_model # Embedding vector size
        self.h = h # Number of heads
        # Make sure d_model is divisible by h
        assert d_model % h == 0, "d_model is not divisible by h"

        self.d_k = d_model // h # Dimension of vector seen by each head
        self.w_q = nn.Linear(d_model, d_model, bias=False) # Wq
        self.w_k = nn.Linear(d_model, d_model, bias=False) # Wk
        self.w_v = nn.Linear(d_model, d_model, bias=False) # Wv
        self.w_o = nn.Linear(d_model, d_model, bias=False) # Wo
        self.dropout = nn.Dropout(dropout)

    @staticmethod
    def attention(query, key, value, mask, dropout: nn.Dropout):
        d_k = query.shape[-1]
        # Just apply the formula from the paper
        # (batch, h, seq_len, d_k) --> (batch, h, seq_len, seq_len)
        attention_scores = (query @ key.transpose(-2, -1)) / math.sqrt(d_k)
        if mask is not None:
            #mask = mask.unsqueeze(1)
            #print('Attention', attention_scores.size())
            #print('Mask', mask.shape)
            mask = mask.view(*mask.shape, *(1,)*(len(attention_scores.size())-mask.ndim)).expand(attention_scores.size())

            attention_scores = attention_scores.masked_fill(mask == 0, -1e9)
            attention_scores.masked_fill_(mask == 0, -1e9)
        attention_scores = attention_scores.softmax(dim=-1) # (batch, h, seq_len, seq_len) # Apply softmax
        if dropout is not None:
            attention_scores = dropout(attention_scores)
        # (batch, h, seq_len, seq_len) --> (batch, h, seq_len, d_k)
        # return attention scores which can be used for visualization
        return (attention_scores @ value), attention_scores

    def forward(self, q, k, v, mask):
        query = self.w_q(q) # (batch, seq_len, d_model) --> (batch, seq_len, d_model)
        key = self.w_k(k) # (batch, seq_len, d_model) --> (batch, seq_len, d_model)
        value = self.w_v(v) # (batch, seq_len, d_model) --> (batch, seq_len, d_model)

        # (batch, seq_len, d_model) --> (batch, seq_len, h, d_k) --> (batch, h, seq_len, d_k)
        query = query.view(query.shape[0], query.shape[1], self.h, self.d_k).transpose(1, 2)
        key = key.view(key.shape[0], key.shape[1], self.h, self.d_k).transpose(1, 2)
        value = value.view(value.shape[0], value.shape[1], self.h, self.d_k).transpose(1, 2)

        # Calculate attention
        x, self.attention_scores = MultiHeadAttentionBlock.attention(query, key, value, mask, self.dropout)

        # Combine all the heads together
        # (batch, h, seq_len, d_k) --> (batch, seq_len, h, d_k) --> (batch, seq_len, d_model)
        x = x.transpose(1, 2).contiguous().view(x.shape[0], -1, self.h * self.d_k)

        # Multiply by Wo
        # (batch, seq_len, d_model) --> (batch, seq_len, d_model)
        return self.w_o(x)

class EncoderBlock(nn.Module):

    def __init__(self, features: int, self_attention_block: MultiHeadAttentionBlock, feed_forward_block: FeedForwardBlock, dropout: float) -> None:
        super().__init__()
        self.self_attention_block = self_attention_block
        self.feed_forward_block = feed_forward_block
        self.residual_connections = nn.ModuleList([ResidualConnection(features, dropout) for _ in range(2)])

    def forward(self, x, src_mask):
        x = self.residual_connections[0](x, lambda x: self.self_attention_block(x, x, x, src_mask))
        x = self.residual_connections[1](x, self.feed_forward_block)
        return x

class Encoder(nn.Module):

    def __init__(self, features: int, layers: nn.ModuleList) -> None:
        super().__init__()
        self.layers = layers
        self.norm = LayerNormalization(features)



    def forward(self, x, mask):
        for layer in self.layers:
            x = layer(x, mask)

        return self.norm(x)

class ClassificationHead(nn.Module):

    def __init__(self, d_model) -> None:
        super().__init__()

        self.avg_pool = nn.AdaptiveAvgPool1d(1)
        # Fully Connected Layer
        self.fc = nn.Linear(d_model, 1)

    def forward(self, x) -> None:

        output = self.avg_pool(x.transpose(1, 2)).squeeze(-1)
        output = self.fc(output)
        return output


class Transformer(nn.Module):

    def __init__(self, encoder: Encoder, src_embed: InputEmbeddings, src_pos: PositionalEncoding, classification_head: ClassificationHead):
        super().__init__()
        self.encoder = encoder
        self.src_embed = src_embed
        self.src_pos = src_pos
        self.classification_head = classification_head

        ## classification head
        self.avg_pool = nn.AdaptiveAvgPool1d(1)
        # Fully Connected Layer to d_model param
        self.fc = nn.Linear(512, 1)

    def encode(self, src, src_mask):
        # (batch, seq_len, d_model)
        src = self.src_embed(src)
        src = self.src_pos(src)
        return self.encoder(src, src_mask)

    def classification(self, x):
        return self.classification_head(x)


def build_transformer(src_vocab_size: int, src_seq_len: int,
                      d_model: int=512, N: int=6, h: int=8,
                      dropout: float=0.1, d_ff: int=2048):
    # Create the embedding layers
    src_embed = InputEmbeddings(d_model, src_vocab_size)

    # Create the positional encoding layers
    src_pos = PositionalEncoding(d_model, src_seq_len, dropout)

    # Create the encoder blocks
    encoder_blocks = []
    for _ in range(N):
        encoder_self_attention_block = MultiHeadAttentionBlock(d_model, h, dropout)
        feed_forward_block = FeedForwardBlock(d_model, d_ff, dropout)
        encoder_block = EncoderBlock(d_model, encoder_self_attention_block, feed_forward_block, dropout)
        encoder_blocks.append(encoder_block)

    # Create the encoder and decoder
    encoder = Encoder(d_model, nn.ModuleList(encoder_blocks))
    classification = ClassificationHead(d_model)


    # Create the transformer
    transformer = Transformer(encoder, src_embed, src_pos, classification)

    # Initialize the parameters
    for p in transformer.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)

    return transformer

In [3]:
dataset = load_dataset("imdb")

# Tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Define the PyTorch dataset class
class HFDataset(data.Dataset):
    def __init__(self, hf_dataset, tokenizer):
        self.hf_dataset = hf_dataset
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.hf_dataset)

    def __getitem__(self, idx):
        example = self.hf_dataset[idx]
        text = example["text"]
        label = example["label"]

        # Tokenize the text
        encoding = self.tokenizer.encode_plus(text, padding='max_length', truncation=True, max_length=128, return_tensors='pt')

        return {
            "input_ids": encoding["input_ids"].squeeze(),
            "attention_mask": encoding["attention_mask"].unsqueeze(0).int(),
            "label": label,
        }

pytorch_dataset = HFDataset(dataset["train"], tokenizer)

# Create a PyTorch DataLoader
batch_size = 64
dataloader = DataLoader(
    pytorch_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=2,
    collate_fn=lambda x: {
        "input_ids": torch.stack([item["input_ids"] for item in x]),
        "attention_mask": torch.stack([item["attention_mask"] for item in x]),
        "label": torch.tensor([item["label"] for item in x])
    },
    pin_memory=True,
)


Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [4]:
src_vocab_size = 200000
#tgt_vocab_size = 5000
#num_classes = 2
#d_model = 512
num_heads = 2 #8
num_layers = 1
#d_ff = 2048
max_seq_length = 512 #128
#dropout = 0.1

model = build_transformer(src_vocab_size, max_seq_length)


In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

criterion = nn.BCEWithLogitsLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)



model.to(device)
model.train()

num_epochs = 6

for epoch in range(num_epochs):
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch + 1, num_epochs))
    print('Training...')
    
    #torch.cuda.empty_cache()
    model.train()
    total_train_loss = 0
    for batch in dataloader:
        #input_ids, attention_mask, labels_batch = [x.to(device) for x in batch]
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        label = batch['label'].to(device)

        optimizer.zero_grad()
        #outputs = model(input_ids, attention_mask=attention_mask, labels=labels_batch)
        #outputs = model(input_ids)

        #output = model(input_ids, attention_mask)
        encoder_output = model.encode(input_ids, attention_mask) # (B, seq_len, d_model)
        output = model.classification(encoder_output).to(device)

        loss = criterion(output.squeeze(), label.float())

        loss.backward()
        optimizer.step()
        
        total_train_loss += loss.item()

    
    avg_train_loss = total_train_loss / len(dataloader) 
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    
    total_eval_loss = 0.0
    model.eval()     # Optional when not using Model Specific layer
    
    print("Running Validation...")
    
    for batch in dataloader:
        
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        label = batch['label'].to(device)
        
        with torch.no_grad():
            encoder_output = model.encode(input_ids, attention_mask) # (B, seq_len, d_model)
            output = model.classification(encoder_output).to(device)
        
        
        loss = criterion(output.squeeze(), label.float())
        total_eval_loss += loss.item()
        
    avg_val_loss = total_eval_loss / len(dataloader)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))


======== Epoch 1 / 6 ========
Training...
  Average training loss: 0.62
Running Validation...
  Validation Loss: 0.33

======== Epoch 2 / 6 ========
Training...
  Average training loss: 0.32
Running Validation...
  Validation Loss: 0.19

======== Epoch 3 / 6 ========
Training...
  Average training loss: 0.20
Running Validation...
  Validation Loss: 0.07

======== Epoch 4 / 6 ========
Training...
  Average training loss: 0.10
Running Validation...
  Validation Loss: 0.04

======== Epoch 5 / 6 ========
Training...
  Average training loss: 0.05
Running Validation...
  Validation Loss: 0.01

======== Epoch 6 / 6 ========
Training...
  Average training loss: 0.02
Running Validation...
  Validation Loss: 0.00
